# Regression using Cyclic Boosting

First, install the  package and its dependencies

```sh
!pip install cyclic-boosting
```

In [ ]:
import pandas as pd
import numpy as np

# Let's use the test dataset from kaggle

Sign in to Kaggle at the URL below and download the dataset.  
https://www.kaggle.com/datasets/yasserh/walmart-dataset

Place the downloaded dataset in the following directory.  
examples/regression/tornado/test_2/Walmart_data/

For time-series data, a "date" column must be included to indicate the date and time the data was obtained. The column name and format must be consistent. The "dayofweek" column for the day of the week and the "dayofyear" column for the total number of days in the year are automatically created if not already present, but if they are already present, the column names must be correct.

This dataset has data for each week.

In [ ]:
df = pd.read_csv("./Walmart_data/Walmart.csv")
df = df.rename(columns={'Date': 'date'})
df["date"] = pd.to_datetime(df["date"], format='%d-%m-%Y')
df.to_csv("./Walmart.csv", index=False)

# Automated Machine Learning with Tornado
With tornado, you can automatically perform data preparation, feature property setting, hyperparameter tuning, model building, training, evaluation, and plotting!

In [ ]:
from cyclic_boosting.tornado import Generator, Manager, Trainer

data_deliverler = Generator.TornadoDataModule("./Walmart.csv")
manager = Manager.TornadoVariableSelectionModule()
trainer = Trainer.SqueezeTrainer(data_deliverler, manager)
trainer.run(target="weekly_sales", log_policy="compute_COD", verbose=False)

# Load the best model and make predictions.

Get the best model path.

In [ ]:
import pickle
from pathlib import Path

model_nos = []
for p in sorted(Path("./models/").glob("model*")):
    model_nos.append(str(p)[str(p).find("_") + 1 :])
model_path = f"./models/model_{model_nos[-1]}/model_{model_nos[-1]}.pkl"
print(model_path)

Make predictions with the best model.

In [ ]:
data = {
    "store": [5],
    "holiday_flag": [0],
    "temperature": [48.3],
    "fuel_price": [2.976],
    "cpi": [211.9560305],
    'unemployment': [6.634],
    "dayofweek": [4],
    "dayofyear": [7],
}
X = pd.DataFrame(data)

with open(model_path, "rb") as f:
    CB_est = pickle.load(f)
    yhat = CB_est.predict(X.copy())
    print(yhat)